## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from configG import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,New Norfolk,AU,-42.7826,147.0587,41.72,75,3,1.99
1,1,Pevek,RU,69.7008,170.3133,39.24,86,7,10.18
2,2,Port Macquarie,AU,-31.4333,152.9167,49.96,85,3,4.70
3,3,Mar Del Plata,AR,-38.0023,-57.5575,50.02,100,90,8.05
4,4,Hobart,AU,-42.8794,147.3294,42.06,79,20,3.44


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
5,5,Corrales,US,35.2378,-106.6067,75.97,45,1,3.00
9,9,Balsas,BR,-7.5325,-46.0356,81.30,41,15,13.58
11,11,Faya,SA,18.3851,42.4509,79.32,41,78,7.72
14,14,Pontianak,ID,-0.0333,109.3333,87.78,79,20,5.75
15,15,Kieta,PG,-6.2167,155.6333,79.61,82,76,8.61
22,22,Souillac,MU,-20.5167,57.5167,75.58,60,40,12.66
24,24,Banda Aceh,ID,5.5577,95.3222,82.33,75,84,11.99
28,28,Korla,CN,41.7597,86.1469,93.02,17,1,14.16
29,29,Mafinga,TZ,-7.2500,35.0667,85.96,27,8,4.32
31,31,Biltine,TD,14.5333,20.9167,89.02,49,85,13.91


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID       263
City          263
Country       263
Lat           263
Lng           263
Max Temp      263
Humidity      263
Cloudiness    263
Wind Speed    263
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID       263
City          263
Country       263
Lat           263
Lng           263
Max Temp      263
Humidity      263
Cloudiness    263
Wind Speed    263
dtype: int64

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
# "Current Description"
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Corrales,US,75.97,35.2378,-106.6067,
9,Balsas,BR,81.30,-7.5325,-46.0356,
11,Faya,SA,79.32,18.3851,42.4509,
14,Pontianak,ID,87.78,-0.0333,109.3333,
15,Kieta,PG,79.61,-6.2167,155.6333,
22,Souillac,MU,75.58,-20.5167,57.5167,
24,Banda Aceh,ID,82.33,5.5577,95.3222,
28,Korla,CN,93.02,41.7597,86.1469,
29,Mafinga,TZ,85.96,-7.2500,35.0667,
31,Biltine,TD,89.02,14.5333,20.9167,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]   
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()  
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")
hotel_df.head()     

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Corrales,US,75.97,35.2378,-106.6067,
9,Balsas,BR,81.30,-7.5325,-46.0356,
11,Faya,SA,79.32,18.3851,42.4509,
14,Pontianak,ID,87.78,-0.0333,109.3333,
15,Kieta,PG,79.61,-6.2167,155.6333,


In [21]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City          263
Country       263
Max Temp      263
Lat           263
Lng           263
Hotel Name    263
dtype: int64

In [22]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))